In [13]:
from kedro.framework.context import load_context
proj_path = '../../../' 
context = load_context(proj_path)
# df = catalog.load("XXX")
parameters = context.params

In [24]:
import pandas as pd
import time
import requests
from bs4 import BeautifulSoup
import re

In [44]:
def exec_scraping_netkeiba_race_results(race_id_list):
    """
    netkeiba.comのレースIDのリストを渡して、それらをまとめて{'レースID', 結果のDataFrame}という形式の辞書型に格納する
    ex. race_results['201901010101']
        -> df 着順 枠番 馬番 馬名 性齢 斤量 騎手 タイム 着差 単勝 人気 馬体重 調教師 horse_id jockey_id course_id
    """
    race_results_dict = {}
    for race_id in race_id_list:
        try:
            # ベース情報を取ってくる
            url = 'https://db.netkeiba.com/race/' + race_id
            race_results_dict[race_id] = pd.read_html(url)[0]
            
            # 詳細情報取得のために、再度対象URLのページを取得する
            html = requests.get(url)
            html.encoding = "EUC-JP"
            soup = BeautifulSoup(html.text, "html.parser")

            # horse_idを取得する
            horse_id_list = []
            horse_a_list = soup.find("table", attrs={"summary": "レース結果"}).find_all("a", attrs={"href": re.compile("^/horse")})
            for a in horse_a_list:
                horse_id = re.findall(r"\d+", a["href"])
                horse_id_list.append(horse_id[0])

            # jockey_idを取得する
            jockey_id_list = []
            jockey_a_list = soup.find("table", attrs={"summary": "レース結果"}).find_all(
                "a", attrs={"href": re.compile("^/jockey")}
            )
            for a in jockey_a_list:
                jockey_id = re.findall(r"\d+", a["href"])
                jockey_id_list.append(jockey_id[0])

            # horse_idとjockey_idをベースに追加
            race_results_dict[race_id]["horse_id"] = horse_id_list
            race_results_dict[race_id]["jockey_id"] = jockey_id_list
      
            # course_idをベースに追加
            race_results_dict[race_id]['course_id'] = [int(race_id[4:6])]*len(horse_id_list)

            # course_len, course_type , weather, race_type, ground_state, dateを取得してベースに追加
            texts = (
                soup.find("div", attrs={"class": "data_intro"}).find_all("p")[0].text
                + soup.find("div", attrs={"class": "data_intro"}).find_all("p")[1].text
            )
            info = re.findall(r'\w+', texts)
            for text in info:
                if text in ["芝", "ダート"]:
                    race_results_dict[race_id]["race_type"] = text
                if "障" in text:
                    race_results_dict[race_id]["race_type"] = "障害"
                if "m" in text:
                    race_results_dict[race_id]["course_len"] = int(re.findall(r"\d+", text)[0])
                if text in ["良", "稍重", "重", "不良"]:
                    race_results_dict[race_id]["ground_state"] = text
                if text in ["曇", "晴", "雨", "小雨", "小雪", "雪"]:
                    race_results_dict[race_id]["weather"] = text
                if "年" in text:
                    race_results_dict[race_id]["date"] = text
                # change コース特性追加
                if "右" in text:
                    race_results_dict[race_id]["course_type"] = "right"
                if "左" in text:
                    race_results_dict[race_id]["course_type"] = "left"
                if "直線" in text:
                    race_results_dict[race_id]["course_type"] = "straight"            

            time.sleep(0.1)

        except IndexError:
            continue
#         except:
#             break
    return race_results_dict

### mainの中身

In [45]:
race_id_list = []

for place in range(1,11):
    for kai in range(1,6):
        for day in range(1,9):
            for r in range(1,13):
                race_id = '2019' + str(place).zfill(2) + str(kai).zfill(2) + str(day).zfill(2) + str(r).zfill(2)
                race_id_list.append(race_id)
# -> ['201901010101', '201901010102', ,,, ]

# スクレイピング実行
race_results_dict = exec_scraping_netkeiba_race_results(race_id_list[0:parameters["scraping_limit"]])

In [46]:
race_results_dict

{'201901010101':    着順  枠番  馬番         馬名  性齢  斤量    騎手     タイム     着差     単勝  ...       調教師  \
 0   1   1   1      ゴルコンダ  牡2  54  ルメール  1:48.3    NaN    1.4  ...  [東] 木村哲也   
 1   2   3   3   プントファイヤー  牡2  54  岩田康誠  1:50.1      大    3.5  ...  [東] 手塚貴久   
 2   3   4   4  ラグリマスネグラス  牡2  51  団野大成  1:50.9      5   46.6  ...  [東] 藤沢和雄   
 3   4   8   9     キタノコドウ  牡2  51  菅原明良  1:51.5  3.1/2   56.8  ...   [東] 高木登   
 4   5   5   5   ネモフィラブルー  牡2  54  川島信二  1:51.7  1.1/2  140.3  ...  [西] 矢作芳人   
 5   6   8   8  マイネルラクスマン  牡2  54  丹内祐次  1:52.1  2.1/2    9.7  ...  [東] 金成貴史   
 6   7   2   2    サンモンテベロ  牝2  54   黛弘人  1:52.5  2.1/2  114.7  ...  [東] 中野栄治   
 7   8   7   7   エスカレーション  牝2  54  藤岡佑介  1:52.5    アタマ   26.1  ...  [東] 高柳瑞樹   
 8   9   6   6   セイウンジュリア  牝2  54  池添謙一  1:52.6     クビ   16.4  ...  [西] 浅見秀一   
 
      horse_id jockey_id course_id course_len  course_type  weather race_type  \
 0  2017105318     05339         1       1800        right        曇         芝   
 1  2017104612     0

In [47]:
# 表として見やすいように、dfのindexにrace idを入れる
for key in race_results_dict.keys():
    race_results_dict[key].index = [key]*len(race_results_dict[key])

# 各レース結果のdfを1つに結合する
race_results_df = pd.concat((race_results_dict[key] for key in race_results_dict.keys()),sort=False)

In [48]:
race_results_df

,着順,枠番,馬番,馬名,性齢,斤量,騎手,タイム,着差,単勝,...,調教師,horse_id,jockey_id,course_id,course_len,course_type,weather,race_type,ground_state,date
201901010101,1,1,1,ゴルコンダ,牡2,54,ルメール,1:48.3,NaN,1.4,...,[東] 木村哲也,2017105318,05339,1,1800,right,曇,芝,良,2019年7月27日
201901010101,2,3,3,プントファイヤー,牡2,54,岩田康誠,1:50.1,大,3.5,...,[東] 手塚貴久,2017104612,05203,1,1800,right,曇,芝,良,2019年7月27日
201901010101,3,4,4,ラグリマスネグラス,牡2,51,団野大成,1:50.9,5,46.6,...,[東] 藤沢和雄,2017103879,01180,1,1800,right,曇,芝,良,2019年7月27日
201901010101,4,8,9,キタノコドウ,牡2,51,菅原明良,1:51.5,3.1/2,56.8,...,[東] 高木登,2017106259,01179,1,1800,right,曇,芝,良,2019年7月27日
201901010101,5,5,5,ネモフィラブルー,牡2,54,川島信二,1:51.7,1.1/2,140.3,...,[西] 矢作芳人,2017104140,01062,1,1800,right,曇,芝,良,2019年7月27日
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201901010208,3,4,4,エンシュラウド,セ3,54,ルメール,1:48.6,1.1/2,2,...,[東] 藤沢和雄,2016110030,05339,1,1800,right,晴,芝,良,2019年7月28日
201901010208,4,2,2,グランツリヒター,牡3,54,三浦皇成,1:49.0,2.1/2,12.3,...,[東] 中川公成,2016103280,01122,1,1800,right,晴,芝,良,2019年7月28日
201901010208,5,7,7,エピローグ,牝3,52,荻野極,1:49.4,2.1/2,19.5,...,[西] 小崎憲,2016101402,01160,1,1800,right,晴,芝,良,2019年7月28日
201901010208,6,3,3,サイモンルチアーノ,牡4,57,黛弘人,1:49.9,3,147.5,...,[東] 田中清隆,2015109012,01109,1,1800,right,晴,芝,良,2019年7月28日
